# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to regularization!

## Summary

You will be able to:

- Build a linear regression model with interactions and polynomial features 
- Use feature selection to obtain the optimal subset of features in a dataset

## Let's Get Started!

Below we import all the necessary packages for this lab.

In [1]:
# Run this cell without changes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Load the data.

In [2]:
# Run this cell without changes

# Load data from CSV
df = pd.read_csv("ames.csv")
# Subset columns
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

# Split the data into X and y
y = df['SalePrice']
X = df.drop(columns='SalePrice')

# Split into train, test, and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, random_state=0)

## Build a Baseline Housing Data Model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Scale all the predictors using `StandardScaler`, then convert these scaled features back into DataFrame objects
- Build a baseline `LinearRegression` model using *scaled variables* as predictors and use the $R^2$ score to evaluate the model 

In [3]:
# select numerical features (not actually necessary since all are numerical =) )
X_train_cont = X_train.select_dtypes(include='number')

# instantiate standard scaler
scaler = StandardScaler()

# fit to & transform training data; also make the output a dataframe with same columns as X_train_cont
X_train = pd.DataFrame(scaler.fit_transform(X_train_cont), columns=X_train_cont.columns)

In [4]:
# Your code here

# Scale X_train and X_test using StandardScaler
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

# Ensure X_train and X_test are scaled DataFrames
# (hint: you can set the columns using X.columns)
print(X_train)
print(X_test)

      LotArea  OverallQual  OverallCond  TotalBsmtSF  1stFlrSF  2ndFlrSF  \
0   -0.114710    -0.099842    -0.509252    -0.639316 -0.804789  1.261552   
1   -0.176719     0.632038    -0.509252     0.838208  0.641608 -0.808132   
2   -0.246336    -0.831723     1.304613    -0.012560 -0.329000 -0.808132   
3   -0.378617    -0.831723     1.304613    -0.339045 -0.609036 -0.808132   
4   -0.010898    -1.563603     1.304613    -2.531499 -1.315922  0.550523   
..        ...          ...          ...          ...       ...       ...   
816 -0.532331    -0.099842    -0.509252    -0.510628 -0.897228 -0.808132   
817 -0.309245    -0.099842    -0.509252     0.514106  0.315353 -0.808132   
818  0.119419     0.632038    -0.509252    -0.513011 -0.899947  1.684999   
819 -0.002718    -0.099842     1.304613    -0.889542 -1.329516  0.783758   
820  0.086287    -0.099842     0.397681     0.433080  0.179414 -0.808132   

     GrLivArea  TotRmsAbvGrd  GarageArea  Fireplaces  
0     0.499114      0.250689    

In [5]:
# Your code here

# Create a LinearRegression model and fit it on scaled training data
# instantiate
linreg = LinearRegression()
linreg.fit(X_train, y_train)

# Calculate a baseline r-squared score on training data
print(f'Baseline R2: {linreg.score(X_train, y_train)}')

Baseline R2: 0.7868344817421309


## Add Interactions

Instead of adding all possible interaction terms, let's try a custom technique. We are only going to add the interaction terms that increase the $R^2$ score as much as possible. Specifically we are going to look for the 7 interaction terms that each cause the most increase in the coefficient of determination.

### Find the Best Interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Create a data structure that stores the pair of columns used as well as the $R^2$ score for each combination.

***Hint:*** We have imported the `combinations` function from `itertools` for you ([documentation here](https://docs.python.org/3/library/itertools.html#itertools.combinations)). Try applying this to the columns of `X_train` to find all of the possible pairs.

Print the 7 interactions that result in the highest $R^2$ scores.

In [6]:
# Your code here

# Set up data structure
R2_pairs = {}

# make pair a key for value R2

# Find combinations of columns and loop over them
for pair in combinations(X_train.columns, 2):
    # Make copies of X_train and X_test
    X_train_c = X_train.copy()
    X_test_c = X_test.copy()
    
    # Add interaction term to data
    X_train_c['interaction'] = X_train_c[pair[0]]*X_train_c[pair[1]]
    X_test_c['interaction'] = X_test_c[pair[0]]*X_test_c[pair[1]]
    
    # Find r-squared score (fit on training data, evaluate on test data)
    lr = LinearRegression()
    lr.fit(X_train_c, y_train)
    R2 = lr.score(X_test_c, y_test)
    
    # Append to data structure
    R2_pairs[pair] = R2
    
# Sort and subset the data structure to find the top 7
top_7_interactions = sorted(R2_pairs.items(), key = lambda kv: kv[1], reverse=True)[:7]

In [7]:
# check results

for pair in top_7_interactions:
    print(R2_pairs[pair[0]])

0.7211105666140569
0.7071649207050108
0.6690980823779027
0.6529699515652585
0.647299489040519
0.6429019879233769
0.6422324294284367


### Add the Best Interactions

Write code to include the 7 most important interactions in `X_train` and `X_test` by adding 7 columns. Use the naming convention `"col1_col2"`, where `col1` and `col2` are the two columns in the interaction.

In [40]:
for pair in top_7_interactions:
    print(pair[0][0])

LotArea
LotArea
LotArea
LotArea
2ndFlrSF
OverallCond
OverallQual


In [8]:
# Your code here

# Loop over top 7 interactions
for pair in top_7_interactions:
    
    # Extract column names from data structure
    col1 = pair[0][0]
    col2 = pair[0][1]
    
    # Construct new column name
    col_name = col1 + '_' + col2
    
    # Add new column to X_train and X_test
    X_train[col_name] = X_train[col1]*X_train[col2]
    X_test[col_name] = X_test[col1]*X_test[col2]

## Add Polynomials

Now let's repeat that process for adding polynomial terms.

### Find the Best Polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of degree 4 with `PolynomialFeatures`, the particular column is raised to the power of 2 and 3 as well in other terms.

We only want to include "pure" polynomials, so make sure no interactions are included.

Once again you should make a data structure that contains the values you have tested. We recommend a list of tuples of the form:

`(col_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [14]:
my_X_train['LotArea']

0     -0.114710
1     -0.176719
2     -0.246336
3     -0.378617
4     -0.010898
         ...   
816   -0.532331
817   -0.309245
818    0.119419
819   -0.002718
820    0.086287
Name: LotArea, Length: 821, dtype: float64

In [10]:
test_poly = PolynomialFeatures(2, include_bias=False)

In [13]:
test_poly.fit_transform(my_X_train[['LotArea']])

array([[-1.14710362e-01,  1.31584671e-02],
       [-1.76719426e-01,  3.12297555e-02],
       [-2.46336462e-01,  6.06816527e-02],
       ...,
       [ 1.19418849e-01,  1.42608615e-02],
       [-2.71773792e-03,  7.38609940e-06],
       [ 8.62873569e-02,  7.44550796e-03]])

In [15]:
# Your code here

# Set up data structure
results = []

# Loop over all columns
for col in X_train.columns:
    # Loop over degrees 2, 3, 4
        for degree in [2, 3, 4]:
        # Make a copy of X_train and X_test
            my_X_train = X_train.copy()
            my_X_test = X_test.copy()
        # Instantiate PolynomialFeatures with relevant degree
            poly = PolynomialFeatures(degree, include_bias=False)
        # Fit polynomial to column and transform column
            poly_col_train = pd.DataFrame(poly.fit_transform(my_X_train[[col]]))
            poly_col_test = pd.DataFrame(poly.transform(my_X_test[[col]]))
        
        # Add polynomial to data
            col_name = f'poly_{col}'
            my_X_train = pd.concat([my_X_train.drop([col], axis=1), poly_col_train], axis=1)
            my_X_test = pd.concat([my_X_test.drop([col], axis=1), poly_col_test], axis=1)
        # Hint: use pd.concat since you're combining two DataFrames
        # Hint: drop the column before combining so it doesn't appear twice
    
        # Find r-squared score
            lr = LinearRegression()
            lr.fit(my_X_train, y_train)
            R2 = lr.score(my_X_test, y_test)
        # Append to data structure
            results.append((col, degree, R2))
# Sort and subset the data structure to find the top 7
top_7 = sorted(results, key = lambda x: x[2], reverse=True)[:7]
print(top_7)

[('GrLivArea', 3, 0.8283344365591123), ('OverallQual_2ndFlrSF', 3, 0.8222074062501774), ('LotArea_Fireplaces', 4, 0.8126598088713186), ('LotArea_Fireplaces', 3, 0.8124111133929935), ('OverallQual', 3, 0.8068023565969222), ('OverallQual_2ndFlrSF', 2, 0.8057607786926376), ('OverallQual', 4, 0.8033455378866856)]


### Add the Best Polynomials

If there are duplicate column values in the results above, don't add multiple of them to the model, to avoid creating duplicate columns. (For example, if column `A` appeared in your list as both a 2nd and 3rd degree polynomial, adding both would result in `A` squared being added to the features twice.) Just add in the polynomial that results in the highest R-Squared.

In [17]:
# Convert to DataFrame for easier manipulation
top_polynomials = pd.DataFrame(top_7, columns=["Column", "Degree", "R^2"])
top_polynomials

,Column,Degree,R^2
0,GrLivArea,3,0.828334
1,OverallQual_2ndFlrSF,3,0.822207
2,LotArea_Fireplaces,4,0.812660
3,LotArea_Fireplaces,3,0.812411
4,OverallQual,3,0.806802
5,OverallQual_2ndFlrSF,2,0.805761
6,OverallQual,4,0.803346


In [18]:
# Drop duplicate columns
top_polynomials.drop_duplicates(subset="Column", inplace=True)
top_polynomials

,Column,Degree,R^2
0,GrLivArea,3,0.828334
1,OverallQual_2ndFlrSF,3,0.822207
2,LotArea_Fireplaces,4,0.812660
4,OverallQual,3,0.806802


In [19]:
# Loop over remaining results
for (col, degree, _) in top_polynomials.values:
    # Create polynomial terms
    poly = PolynomialFeatures(degree, include_bias=False)
    col_transformed_train = pd.DataFrame(poly.fit_transform(X_train[[col]]),
                                        columns=poly.get_feature_names([col]))
    col_transformed_test = pd.DataFrame(poly.transform(X_test[[col]]),
                                    columns=poly.get_feature_names([col]))
    # Concat new polynomials to X_train and X_test
    X_train = pd.concat([X_train.drop(col, axis=1), col_transformed_train], axis=1)
    X_test = pd.concat([X_test.drop(col, axis=1), col_transformed_test], axis=1)

In [72]:
# Your code here

# Filter out duplicates
# add OverallQual_2ndFlrSF with degree 2, OverallQual with degree 2, and 2ndFlrSF_TotRmsAbvGrd with degree 2
# for col in [x[0] for x in top_7]:

# Loop over remaining results
poly = PolynomialFeatures(2)
for col in ['OverallQual_2ndFlrSF', 'OverallQual', '2ndFlrSF_TotRmsAbvGrd']:
    # Create polynomial terms
    col_name = f'poly_{col}'
    X_train[col_name] = pd.DataFrame(poly.fit_transform(X_train[[col]])).loc[:,2]
    X_test[col_name] = pd.DataFrame(poly.transform(X_test[[col]])).loc[:,2]
    # Concat new polynomials to X_train and X_test
    

Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [20]:
# Your code here
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 821 entries, 0 to 820
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   LotArea                  821 non-null    float64
 1   OverallCond              821 non-null    float64
 2   TotalBsmtSF              821 non-null    float64
 3   1stFlrSF                 821 non-null    float64
 4   2ndFlrSF                 821 non-null    float64
 5   TotRmsAbvGrd             821 non-null    float64
 6   GarageArea               821 non-null    float64
 7   Fireplaces               821 non-null    float64
 8   LotArea_1stFlrSF         821 non-null    float64
 9   LotArea_TotalBsmtSF      821 non-null    float64
 10  LotArea_GrLivArea        821 non-null    float64
 11  2ndFlrSF_TotRmsAbvGrd    821 non-null    float64
 12  OverallCond_TotalBsmtSF  821 non-null    float64
 13  GrLivArea                821 non-null    float64
 14  GrLivArea^2              8

In [21]:
X_train.head()

,LotArea,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,TotRmsAbvGrd,GarageArea,Fireplaces,LotArea_1stFlrSF,LotArea_TotalBsmtSF,...,OverallQual_2ndFlrSF,OverallQual_2ndFlrSF^2,OverallQual_2ndFlrSF^3,LotArea_Fireplaces,LotArea_Fireplaces^2,LotArea_Fireplaces^3,LotArea_Fireplaces^4,OverallQual,OverallQual^2,OverallQual^3
0,-0.114710,-0.509252,-0.639316,-0.804789,1.261552,0.250689,0.327629,-0.994820,0.092318,0.073336,...,-0.125956,0.015865,-0.001998,0.114116,0.013022,0.001486,1.695855e-04,-0.099842,0.009968,-0.000995
1,-0.176719,-0.509252,0.838208,0.641608,-0.808132,-0.365525,0.079146,-0.994820,-0.113385,-0.148128,...,-0.510770,0.260886,-0.133253,0.175804,0.030907,0.005434,9.552459e-04,0.632038,0.399472,0.252481
2,-0.246336,1.304613,-0.012560,-0.329000,-0.808132,-0.981739,-1.105931,-0.994820,0.081045,0.003094,...,0.672141,0.451774,0.303656,0.245060,0.060055,0.014717,3.606557e-03,-0.831723,0.691762,-0.575354
3,-0.378617,1.304613,-0.339045,-0.609036,-0.808132,-0.981739,-1.134602,0.588023,0.230591,0.128368,...,0.672141,0.451774,0.303656,-0.222636,0.049567,-0.011035,2.456852e-03,-0.831723,0.691762,-0.575354
4,-0.010898,1.304613,-2.531499,-1.315922,0.550523,0.250689,-2.281450,-0.994820,0.014341,0.027589,...,-0.860799,0.740974,-0.637829,0.010842,0.000118,0.000001,1.381725e-08,-1.563603,2.444854,-3.822780


## Full Model R-Squared

Check out the $R^2$ of the full model with your interaction and polynomial terms added. Print this value for both the train and test data.

In [22]:
# Your code here
linreg = LinearRegression()
linreg.fit(X_train, y_train)
R2_train = linreg.score(X_train, y_train)
R2_test = linreg.score(X_test, y_test)
print(f'Train: {R2_train}')
print(f'Test: {R2_test}')

Train: 0.8571817758242435
Test: 0.6442143449159354


It looks like we may be overfitting some now. Let's try some feature selection techniques.

## Feature Selection

First, test out `RFE` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) with several different `n_features_to_select` values. For each value, print out the train and test $R^2$ score and how many features remain.

In [24]:
# Your code here

# iterable for number of features, to test several
features_to_select = [5, 10, 15]

# loop over different choices of number of features to select
for n in features_to_select:
    # instantiate estimator (recursive feature elimination estimator)
    rfe = RFE(LinearRegression(), n_features_to_select=n)
    
    # fit to training data
    X_train_rfe = rfe.fit_transform(X_train, y_train)
    X_test_rfe = rfe.transform(X_test)
    
    # get R2 score for a linear regression with selected features
    # instantiate linear regression
    lr = LinearRegression()
    # fit to training data with selected features
    lr.fit(X_train_rfe, y_train)
    # get R2 scores
    R2_train = lr.score(X_train_rfe, y_train)
    R2_test = lr.score(X_test_rfe, y_test)
    
    # print message
    print(f'Features remaining: {n}')
    print(f'Train score: {R2_train}')
    print(f'Test score: {R2_test}')
    print('\n')

Features remaining: 5
Train score: 0.776039994126505
Test score: 0.6352981725272361


Features remaining: 10
Train score: 0.8191862278324273
Test score: 0.6743476159860744


Features remaining: 15
Train score: 0.8483321237427194
Test score: 0.7040137671087123




Now test out `Lasso` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)) with several different `alpha` values.

In [27]:
# Your code here
alphas = [1, 10, 100, 1000, 10000]

results = []

for alpha in alphas:
    # instantiate lasso regression with current alpha
    lasso = Lasso(alpha=alpha)
    # fit lasso regression model to training data
    lasso.fit(X_train, y_train)
    # append R2 score on test data to list
    R2 = lasso.score(X_test, y_test)
    results.append((alpha, R2))
    # print coefficients used
    print(f"Using {sum(abs(lasso.coef_) >= 10**(-10))} out of {X_train.shape[1]} features")
    print(f'R2 = {R2}')
    print('\n')
    
sorted(results, key = lambda x: x[1], reverse=True)

Using 26 out of 26 features
R2 = 0.6485699116355159


Using 26 out of 26 features
R2 = 0.6480527180183046


Using 25 out of 26 features
R2 = 0.6471042867008621


Using 17 out of 26 features
R2 = 0.72222786778698


Using 12 out of 26 features
R2 = 0.7939567393897815




[(10000, 0.7939567393897815),
 (1000, 0.72222786778698),
 (1, 0.6485699116355159),
 (10, 0.6480527180183046),
 (100, 0.6471042867008621)]

Compare the results. Which features would you choose to use?

Lasso seems to work best here, with 12 out of 26 features and an alpha of 10,000.

## Evaluate Final Model on Validation Data

### Data Preparation

At the start of this lab, we created `X_val` and `y_val`. Prepare `X_val` the same way that `X_train` and `X_test` have been prepared. This includes scaling, adding interactions, and adding polynomial terms.

In [28]:
# Your code here

'''apply standard scaler'''
X_val = pd.DataFrame(scaler.transform(X_val), columns=X_val.columns)

'''add interaction terms'''
# Loop over top 7 interactions
for pair in top_7_interactions:
    # Extract column names from data structure
    col1 = pair[0][0]
    col2 = pair[0][1]
    # Construct new column name
    col_name = col1 + '_' + col2
    # Add new column to X_val
    X_val[col_name] = X_val[col1]*X_val[col2]

'''add polynomial terms'''
for (col, degree, _) in top_polynomials.values:
    poly = PolynomialFeatures(degree, include_bias=False)
    col_transformed_val = pd.DataFrame(poly.fit_transform(X_val[[col]]),
                                        columns=poly.get_feature_names([col]))
    X_val = pd.concat([X_val.drop(col, axis=1), col_transformed_val], axis=1)

### Evaluation

Using either `RFE` or `Lasso`, fit a model on the complete train + test set, then find R-Squared and MSE values for the validation set.

In [29]:
# Your code here
lasso = Lasso(alpha=10000)
lasso.fit(pd.concat([X_train, X_test]), pd.concat([y_train, y_test]))

print(f'R2: {lasso.score(X_val, y_val)}')
print(f'MSE: {mean_squared_error(y_val, lasso.predict(X_val))}')

R2: 0.7991273457324125
MSE: 1407175023.3081572


## Level Up Ideas (Optional)

### Create a Lasso Path

From this section, you know that when using `Lasso`, more parameters shrink to zero as your regularization parameter goes up. In scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for Subset Selection

This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to apply concepts of bias-variance tradeoff using extensions to linear models and feature selection.